In [ ]:
import mysql.connector
from mysql.connector import Error
import csv
from unidecode import unidecode
from tqdm import tqdm
import time
from datetime import datetime
from query_table import *


def conversione_date(dizionario, k):
    for row in dizionario:
        if row.get(k) != '':
            row[k] = datetime.strptime(row.get(k).strip(), "%B %d, %Y").strftime("%Y-%m-%d")
    return dizionario


def conversione_time(dizionario, k):
    lista_minuti = []
    for row in dizionario:
        if row['type'] == 'Movie':
            for i in range(len(row[k])):
                if row[k][i].isnumeric():
                    lista_minuti.append(row[k][i])
                else:
                    row[k] = ''.join(lista_minuti)
                    lista_minuti = []
                    break
    return dizionario


def menu():
    return int(input('----------------------------------------------------\n'
                     f'1: Caricamento DATI su DATABASE\n2: Aggiunta nuovi valori su DATABASE\n3: Interrogazione time\n'
                     f'4: Update ROW\n5: Drop COLUMN\n6: Exit\n7: Attivazione Trigger\n8: CSV di ritorno\n'
                     '----------------------------------------------------\n'))


def add_row(query, elementi_da_aggiungere, connection):
    cursor = connection.cursor()
    chiavi = []
    for row in elementi_da_aggiungere:
        try:
            cursor.execute(query, row)
            last_insert_id = cursor.lastrowid
            chiavi.append(last_insert_id)
            # print(chiavi)
            connection.commit()
            print("Query successful")
        except Error as err:
            print(f"Error: '{err}'")
    return chiavi


def update_row(query, connection, q_u):
    print('Quale tabella si desidera modificare?')
    tabelle_db = ['attori_cast', 'film', 'partecipazione_attore', 'direttori', 'partecipazione_direttore']
    for i in range(len(tabelle_db)):
        print(f'{i + 1}: {tabelle_db[i].title()}')
    print('----------------------------------------------------')
    sc = int(input('Inserire il numero della tabella da modificare: '))
    print('----------------------------------------------------')
    cursor = connection.cursor()
    try:
        cursor.execute(query, (tabelle_db[sc - 1],))
        results = cursor.fetchall()
        print("Query successful")
        for i in range(len(results)):
            print(f'{i + 1}: {results[i][0].title()}')
        print('----------------------------------------------------')
        op = int(input(f'Quale colonna si desidera modificare?: '))
        print('----------------------------------------------------')
        print(
            f'{q_u[0:8]}{tabelle_db[sc - 1]}{q_u[10:15]}{results[op - 1][0]}{q_u[17:22]}{q_u[22:29]}{results[op - 1][0]}{q_u[31:]}')
        cursor.execute(
            f"UPDATE {tabelle_db[sc - 1]} SET {results[op - 1][0]} = '{input()}' WHERE {results[op - 1][0]} = '{input()}';")
        connection.commit()
    except Error as err:
        print(f"Error: '{err}'")


def delete_row(query, connection):
    print("In quale tabella si desidera svolgere l'operazione?")
    print('----------------------------------------------------')
    tabelle_db = ['attori_cast', 'film', 'partecipazione_attore', 'direttori', 'partecipazione_direttore']
    for i in range(len(tabelle_db)):
        print(f'{i + 1}: {tabelle_db[i].title()}')
    print('----------------------------------------------------')
    sc = int(input('Inserire il numero della tabella da modificare: '))
    cursor = connection.cursor()
    try:
        cursor.execute(query, (tabelle_db[sc - 1],))
        results = cursor.fetchall()
        print("Query successful")
        for i in range(len(results)):
            print(f'{i + 1}: {results[i][0].title()}')
        print('----------------------------------------------------')
        op = int(input(f'Quale colonna si desidera droppare?: '))
        print('----------------------------------------------------')
        cursor.execute(f"ALTER TABLE {tabelle_db[sc - 1]} DROP COLUMN {results[op - 1][0]}")
        connection.commit()
    except Error as err:
        print(f"Error: '{err}'")


def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection


def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")


def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")


def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection


def dati(path):
    with open(path, encoding='utf-8') as file:
        lettore = csv.DictReader(file, delimiter=',')
        dizionario = list(lettore)
    return dizionario


def chiave_numerata(dizionario, chiave, query, connection):
    set_vuoto = {unidecode(persona.strip().lower()) for elem in dizionario for persona in elem[chiave].split(',')
                 if ' ' in persona.strip()}
    # senza fare il lower alcuni nomi identici ma con camel case rimangono
    # all'interno del set e vengono caricati doppi nel dataset, me ne sono accorto quando ho provato a impostare il campo nome come unique
    cursor = connection.cursor()
    lista_title = [(elem.title(),) for elem in set_vuoto]
    try:
        cursor.executemany(query, lista_title)
        connection.commit()
        print("Query successful")
        return [elem for elem in enumerate(set_vuoto, start=1)]
    except Error as err:
        print(f"Error: '{err}'")


def chiave_numerata_piu_chiavi(dizionario, chiave, query, connection):
    cursor = connection.cursor()
    lista_film = [[unidecode(elem[k]) for k in chiave] for elem in dizionario if elem != '']
    try:
        cursor.executemany(query, lista_film)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")


def chiavi_doppie(dizionario, chiave, p, query, connection):
    cursor = connection.cursor()
    lista_chiavi_fk = [(int(persona[0]), int(elem['show_id'][1:])) for persona in p for elem in dizionario if
                       persona[1] in elem[chiave].lower()]
    try:
        cursor.executemany(query, lista_chiavi_fk)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")


def ricostruzione_csv(lunghezza, header):
    lista_liste = []
    lista_tot = []
    for j in range(lunghezza):
        for i in range(len(header)):
            if header[i] != 'show_id':
                lista_liste.append([header[i], []])
            else:
                lista_liste.append([header[i], j + 1])
        lista_tot.append(lista_liste)
        lista_liste = []
    return lista_tot


def dizionarizzazione_csv(input):
    output = [dict(lista) for lista in input]
    for elem in output:
        for k in elem:
            if type(elem[k]) == list:
                elem[k] = str(elem[k])[1:-1].strip()
    return output


def scrittura(diz, fields):
    print(diz)
    with open('csv.finale_netflix', 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, delimiter=',', fieldnames=fields)
        writer.writeheader()
        for row in diz:
            writer.writerow(row)

In [ ]:
import csv
from estrazione_country import *

import numpy as np
import pandas as pd
from unidecode import unidecode

from funzioni import *

create_database_query = 'CREATE DATABASE jetair'
drop_database_query = 'DROP DATABASE IF EXISTS jetair'

connessione = create_server_connection('localhost', 'root', 'Gondoliere01_')

drop_database(connessione, drop_database_query)

create_database(connessione, create_database_query)

connessione_db = create_db_connection('localhost', 'root', 'Gondoliere01_', 'jetair')

create_flight_data = '''
CREATE TABLE flight_data(
    id INT PRIMARY KEY,
    codice_aereoporto_di_partenza INT,
    paese_di_partenza INT,
    codice_aereoporto_di_arrivo INT,
    paese_di_arrivo INT,
    modello_aereo VARCHAR(150),
    numero_di_compagnie VARCHAR(10),
    nome_compagnia VARCHAR(120),
    codice_volo CHAR(6),
    ora_partenza VARCHAR (255),
    ora_arrivo VARCHAR (255),
    num_scali INT,
    prezzo DECIMAL(7,2),
    emissioni_di_co2 DECIMAL(10,2),
    media_co2_percorso DECIMAL(10,2),
    percentuale_co2 INT,
    durata VARCHAR (255),
    mese INT,
    Mese_str VARCHAR(15),
    FOREIGN KEY (codice_aereoporto_di_partenza) REFERENCES airports(id) ON DELETE RESTRICT,
    FOREIGN KEY (codice_aereoporto_di_arrivo) REFERENCES airports(id) ON DELETE RESTRICT,
    FOREIGN KEY (paese_di_partenza) REFERENCES country(id) ON DELETE RESTRICT,
    FOREIGN KEY (paese_di_arrivo) REFERENCES country(id) ON DELETE RESTRICT
    );
    '''

create_airlines = '''
CREATE TABLE airlines(
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100),
    country INT,
    FOREIGN KEY (country) REFERENCES country(id)
    );
    '''

create_airports = '''
CREATE TABLE airports(
    id INT PRIMARY KEY AUTO_INCREMENT,
    airport_code VARCHAR(10) UNIQUE,
    airport_name VARCHAR(100) NOT NULL,
    city_name VARCHAR(100) NOT NULL,
    country_name INT,
    country_code CHAR(3),
    latitude INT,
    longitude INT,
    FOREIGN KEY (country_name) REFERENCES country(id)
    );
    '''

create_fk_flight_data_airlines = '''
CREATE TABLE flight_data_airlines(
    id INT,
    airlines_flight_data INT,
    airlines_airlines INT,
    PRIMARY KEY (id, airlines_flight_data, airlines_airlines),
    FOREIGN KEY (airlines_flight_data) REFERENCES flight_data(id) ON DELETE RESTRICT,
    FOREIGN KEY (airlines_airlines) REFERENCES airlines(id) ON DELETE RESTRICT
    );
    '''

create_fk_rotte1 = '''
CREATE TABLE rotte1(
    airline_id INT,
    departure_airport VARCHAR(10),
    departure_airport_id INT,
    arrival_airport VARCHAR(10),
    arrival_airport_id INT,
    PRIMARY KEY (airline_id, departure_airport, arrival_airport),
    FOREIGN KEY (departure_airport) REFERENCES airports(airport_code),
    FOREIGN KEY (arrival_airport) REFERENCES airports(airport_code),
    FOREIGN KEY (airline_id) REFERENCES airlines(id)
);
'''

create_country = '''
CREATE TABLE country(
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) UNIQUE
    );
    '''

create_table_utenti = '''
CREATE TABLE utenti(
    id INT PRIMARY KEY AUTO_INCREMENT,
    email VARCHAR(120),
    password VARCHAR(30),
    nome VARCHAR(50),
    cognome VARCHAR(50),
    indirizzo VARCHAR(150),
    città VARCHAR(150)
    );
    '''

create_trigger_ins_att = '''
CREATE TRIGGER tr_utenti AFTER INSERT ON utenti
FOR EACH ROW
BEGIN
  INSERT INTO platform (operazione, id, email, password, nome, cognome, indirizzo, città)
  VALUES ('INSERT', NEW.id, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR);
END;
'''

execute_query(connessione_db, create_country)
execute_query(connessione_db, create_airlines)
execute_query(connessione_db, create_airports)
execute_query(connessione_db, create_flight_data)
execute_query(connessione_db, create_fk_flight_data_airlines)
execute_query(connessione_db, create_fk_rotte1)
execute_query(connessione_db, create_table_utenti)

q1 = f'INSERT INTO airports (airport_code, airport_name, city_name, country_name, country_code, latitude, longitude) VALUES (%s, %s, %s, %s, %s, %s, %s)'
q2 = f'INSERT INTO airlines(name, country) VALUES (%s, %s)'
q3 = (
    f'INSERT INTO flight_data(id, codice_aereoporto_di_partenza,paese_di_partenza,codice_aereoporto_di_arrivo,paese_di_arrivo,modello_aereo,'
    f'numero_di_compagnie,nome_compagnia,codice_volo,ora_partenza,ora_arrivo,num_scali,prezzo,emissioni_di_co2,media_co2_percorso,percentuale_co2,durata,mese,Mese_str)'
    f'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)')
q4 = f'INSERT INTO flight_data_airlines(id, airlines_flight_data, airlines_airlines) VALUES (%s, %s, %s)'
q5 = f'INSERT INTO rotte1(airline_id, departure_airport, departure_airport_id, arrival_airport, arrival_airport_id) VALUES (%s, %s, %s, %s, %s)'
q6 = f'INSERT INTO country(name) VALUES (%s)'
q7 = f'INSERT INTO utenti(email, password, nome, cognome, indirizzo, citta) VALUES (%s, %s, %s, %s, %s, %s)'

df_airports = pd.read_csv('../csv_puliti/airports-code2_final.csv')
df_airports.pop('Unnamed: 0')
df_airports.pop('coordinates')
df_airports['Country Name'] = df_airports['Country Name'].map(d)
df_airports = df_airports.replace({np.nan: None})

with open('../csv_puliti/flight_data_final.csv') as file:
    lettore = csv.reader(file)
    next(lettore)
    lista_flight_data = list(lettore)
with open('../csv_puliti/partenze_flight_airports.csv') as file:
    lettore = csv.reader(file)
    next(lettore)
    lista_partenze = list(lettore)
with open('../csv_puliti/arrivi_flight_airports.csv') as file:
    lettore = csv.reader(file)
    next(lettore)
    lista_arrivi = list(lettore)

i = 0
for elem in lista_flight_data:
    i += 1
    elem[0] = i
    elem[1] = lista_partenze[i - 1][1]
    elem[3] = lista_arrivi[i - 1][1]
    elem[11] = int(elem[11])
    elem[17] = int(elem[17])
    elem[12] = float(elem[12])
    try:
        elem[13] = float(elem[13])
    except ValueError:
        elem[13] = float(0)
    try:
        elem[14] = float(elem[14])
    except ValueError:
        elem[14] = float(elem[14] + '.00')
    try:
        elem[15] = int(elem[15])
    except ValueError:
        elem[15] = 0
for elem in lista_flight_data:
    if elem[2] in d.keys():
        elem[2] = d[elem[2]]
    if elem[4] in d.keys():
        elem[4] = d[elem[4]]

with open('flight_data_database_FK.csv', 'w', newline='') as f:
    fieldnames = ['id', 'codice_aereoporto_di_partenza', 'paese_di_partenza', 'codice_aereoporto_di_arrivo',
                  'paese_di_arrivo', 'modello_aereo', 'numero_di_compagnie', 'nome_compagnia', 'codice_volo',
                  'ora_partenza', 'ora_arrivo', 'num_scali', 'prezzo', 'emissioni_di_co2', 'media_co2_percorso',
                  'percentuale_co2', 'durata', 'mese', 'Mese_str']
    writer = csv.writer(f, delimiter=',')
    writer.writerow(fieldnames)
    for row in lista_flight_data:
        writer.writerow(row)

df_airlines = pd.read_csv('../csv_puliti/airlines_final.csv')
df_airlines.pop('Unnamed: 0')
df_airlines['Country'] = df_airlines['Country'].map(d)
df_airlines = df_airlines.replace({np.nan: None})

# df_flight_data = pd.read_csv('../csv_puliti/flight_data_final.csv')
# df_flight_data.pop('Unnamed: 0')
# df_flight_data = df_flight_data.replace({np.nan: None})
# df_flight_data['num_scali'] = df_flight_data['num_scali'].astype(str)
# df_flight_data['mese'] = df_flight_data['mese'].astype(str)
# df_flight_data = df_flight_data.astype(str)
# print(df_flight_data.dtypes)

with open('../csv_puliti/df_da_esportare_final.csv') as file:
    lettore = csv.reader(file)
    next(lettore)
    lista_flight_airlines = list(lettore)
for elem in lista_flight_data:
    elem[0] = int(elem[0])
    elem[1] = int(elem[2])
    elem[2] = int(elem[2])

# df_air_fli = pd.read_csv('../csv_puliti/df_da_esportare_final.csv')
# df_air_fli.pop('Unnamed: 0')
# df_air_fli.drop_duplicates(inplace=True)

df_rotte = pd.read_csv('../csv_puliti/routes_final.csv')
df_rotte.pop('Unnamed: 0')
df_rotte.pop('Airline')
df_rotte.replace(r'\N', np.nan, inplace=True)
df_rotte.dropna(inplace=True)

df_country = pd.read_csv('../csv_puliti/country.csv')
df_country.pop('Unnamed: 0')
df_country = df_country.replace({np.nan: None})

caricamento_dataframe(connessione_db, q6, df_country)
caricamento_dataframe(connessione_db, q1, df_airports)
caricamento_dataframe(connessione_db, q2, df_airlines)
caricamento_lista(connessione_db, q3, lista_flight_data)
caricamento_lista(connessione_db, q4, lista_flight_airlines)
caricamento_dataframe2(connessione_db, q5, df_rotte)

In [ ]:
import pandas as pd

df = pd.read_csv('csv_puliti/flight_data_final.csv')
df.drop_duplicates()
lista = df['nome_compagnia'].to_list()

df_airline = pd.read_csv('csv_puliti/airlines_final.csv')
df_airline.drop_duplicates()
lista_compagnie = df_airline['Name'].to_list()
diz_compagnie = {elem[1]: elem[0] for elem in enumerate(lista_compagnie)}

lista2 = []
for elem in lista:
    lista2.append(elem.replace('[', '').replace(']', '').split('| '))

lista_id = []
for elem in enumerate(lista2, start=1):
    for i in range(len(elem[1])):
        lista_id.append([elem[0], elem[1][i]])

df_da_esportare = pd.DataFrame(lista_id)

df_da_esportare[1] = df_da_esportare[1].map(diz_compagnie)
print(df_da_esportare.isnull().sum())

df_da_esportare.to_csv('df_da_esportare333.csv')

In [ ]:
import pandas as pd

df = pd.read_csv('../csv_puliti/airports-code2_final.csv')
df2 = pd.read_csv('../csv_puliti/airlines_final.csv')
lista_country = df['Country Name'].to_list()
lista_country2 = df2['Country'].to_list()
df3 = pd.read_csv('../csv_puliti/flight_data_database.csv')
lista_country3 = df3['paese_di_partenza'].to_list()
lista_country4 = df3['paese_di_arrivo'].to_list()

lista_country.extend(elem for elem in lista_country2 if type(elem) == str)
lista_country.extend(elem for elem in lista_country3 if type(elem) == str)
lista_country.extend(elem for elem in lista_country4 if type(elem) == str)
s = set()
for elem in lista_country:
    s.add(elem.title().strip())
l = list(s)
d = {l[i] :i+1 for i in range(len(l))}

print(len(s))
df_c = pd.DataFrame(s)
df_c[0].to_csv('country.csv')

In [ ]:
import pandas as pd

df = pd.read_csv('csv_puliti/flight_data_database.csv')
df.drop_duplicates()
df2 = pd.read_csv('csv_puliti/airports_database.csv')
df2.drop_duplicates()

lista1 = df['codice_aereoporto_di_partenza'].to_list()
lista2 = df['codice_aereoporto_di_arrivo'].to_list()
lista3 = df2['airport_code'].to_list()
diz = {lista3[i]: i for i in range(len(lista3))}


lista_flight = [[lista1[i], lista2[i]] for i in range(len(lista1))]

df3 = pd.DataFrame(lista_flight, columns=['partenza', 'arrivo'])
print(df3)
df3['partenza'] = df3['partenza'].map(diz)
df3['arrivo'] = df3['arrivo'].map(diz)
print(df3)
df3['arrivo'].to_csv('arrivi_flight_airports.csv')
df3['partenza'].to_csv('partenze_flight_airports.csv')